In [1]:
from datasets import load_dataset
raw_data = load_dataset("text",data_files="./data_dir.txt")

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(raw_data):
    # 这里可以添加多种操作，不光是tokenize
    # 这个函数处理的对象，就是Dataset这种数据类型，通过features中的字段来选择要处理的数据
    tokenizer = AutoTokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token
    # return tokenizer(raw_data["text"], truncation=True)
    return tokenizer(raw_data["text"], padding=True, truncation=True)

tokenized_datasets = raw_data.map(tokenize_function, batched=True)
print(tokenized_datasets)

# prepair data collator
# from transformers import DataCollatorWithPadding
# collator = DataCollatorWithPadding(tokenizer=tokenizer)

from transformers import DataCollatorForLanguageModeling
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=False)


from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',          # 输出目录
    num_train_epochs=1,              # 训练轮数
    per_device_train_batch_size=1,   # 每个设备上的训练批次大小
)

from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("gpt2")


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collator,
    train_dataset=tokenized_datasets["train"],
    tokenizer=tokenizer, # 不加会报错
)

trainer.train()

trainer.save_model(output_dir = "./model/gpt2")



ModuleNotFoundError: No module named 'datasets'

In [3]:
import torch
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
batch_sentences = ["Hello I'm a single sentence",
                   "And another sentence",
                   "And the very very last one"]
batch = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")
print(batch)

ModuleNotFoundError: No module named 'torch'

好例子

In [ ]:
# https://zhuanlan.zhihu.com/p/147144376
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel


# 从下载好的文件夹中加载tokenizer
# 这里你需要改为自己的实际文件夹路径
tokenizer = GPT2Tokenizer.from_pretrained('./gpt2')

# text = 'Who was Jim Henson ? Jim Henson was a'
text = 'Hello, my name is Deming, give me a nickname:'

# 编码一段文本
# 编码后为[8241, 373, 5395, 367, 19069, 5633, 5395, 367, 19069, 373, 257]
indexed_tokens = tokenizer.encode(text)
print("indexed_tokens: " , indexed_tokens)

# 转换为pytorch tensor
# tensor([[ 8241,   373,  5395,   367, 19069,  5633,  5395,   367, 19069,   373, 257]])
# shape为 torch.Size([1, 11])
tokens_tensor = torch.tensor([indexed_tokens])
print("tokens_tensor: " , tokens_tensor)
# 从下载好的文件夹中加载预训练模型
model = GPT2LMHeadModel.from_pretrained('./gpt2')

# 设置为evaluation模式，去取消激活dropout等模块。
# 在huggingface/transformers框架中，默认就是eval模式
model.eval()

# 预测所有token
with torch.no_grad():
    # 将输入tensor输入，就得到了模型的输出，非常简单
    # outputs是一个元组，所有huggingface/transformers模型的输出都是元组
    # 本初的元组有两个，第一个是预测得分（没经过softmax之前的，也叫作logits），
    # 第二个是past，里面的attention计算的key value值
    # 此时我们需要的是第一个值
    outputs = model(tokens_tensor)
    # predictions shape为 torch.Size([1, 11, 50257])，
    # 也就是11个词每个词的预测得分（没经过softmax之前的）
    # 也叫做logits
    predictions = outputs[0]

# 我们需要预测下一个单词，所以是使用predictions第一个batch，最后一个词的logits去计算
# predicted_index = 582，通过计算最大得分的索引得到的
predicted_index = torch.argmax(predictions[0, -1, :]).item()
# 反向解码为我们需要的文本
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
# 解码后的文本：'Who was Jim Henson? Jim Henson was a man'
# 成功预测出单词 'man'
print(predicted_text)

In [ ]:
gpt2回答： https://huggingface.co/docs/transformers/main/en/model_doc/gpt2#transformers.GPT2ForQuestionAnswering.forward.example

In [ ]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('./gpt2')
tokenizer.pad_token = tokenizer.eos_token  # 指定填充标记(pad_token)使用结束标记(eos_token).指定使用 [SEP] 标记来进行补充填充,而不是默认的 [PAD] 标记。
batch_sentences = ["Hello I'm a single sentence",
                   "And another sentence",
                   "And the very very last one"]

// 填充（padding）和截断（truncation）
batch = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")
print(batch)


例子

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

def generate_text(text):
    encoded_input = tokenizer.encode(text, add_special_tokens=True)
    input_ids = torch.tensor(encoded_input).unsqueeze(0)
    outputs = model.generate(input_ids, max_length=50, num_return_sequences=3, past=past)
    past = outputs.past
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

input_text = "PyTorch是一种用于科学计算的开源机器学习库，可以作为NumPy的替代品，广泛应用于深度学习领域。"

generated_text = generate_text(input_text)
print(generated_text)


例子

In [ ]:
# 加载模型
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


# 加载数据
from datasets import load_dataset
​data = load_dataset('text', data_files='my_data/*.txt')

def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_data = data.map(tokenize_function, batched=True)

# 训练
from transformers import Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir='./results',          # 输出目录
    num_train_epochs=1,              # 训练轮数
    per_device_train_batch_size=1,   # 每个设备上的训练批次大小
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
)
​
trainer.train()

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2", padding_size="left")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(raw_data):
    tokenizer = AutoTokenizer.from_pretrained('gpt2', padding_size="left")
    tokenizer.pad_token = tokenizer.eos_token
    # return tokenizer(raw_data["text"], truncation=True)
    return tokenizer(raw_data["text"], padding=True, truncation=True)

tokenized_datasets = raw_data.map(tokenize_function, batched=True)
print(tokenized_datasets)

# prepair data collator
# from transformers import DataCollatorWithPadding
# collator = DataCollatorWithPadding(tokenizer=tokenizer)
from transformers import DataCollatorForLanguageModeling
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=False)


from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',          # 输出目录
    num_train_epochs=3,              # 训练轮数
    per_device_train_batch_size=5,   # 每个设备上的训练批次大小
    
#     warmup_steps (int，可选，默认为 0):
# 用于从 0 线性预热到 learning_rate 的步数。会覆盖 warmup_ratio 的任何效果。
    warmup_steps=100,           # 
    max_steps=20,             # 如果设置为正数，则要执行的总训练步骤数。会覆盖 num_train_epochs。如果使用有限的可迭代数据集，在耗尽所有数据时，训练可能会在达到设置的步数之前停止。
    learning_rate=3e-4          # 学习率123123
)

from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("gpt2")


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collator,
    train_dataset=tokenized_datasets["train"],
    tokenizer=tokenizer, 
)

trainer.train()

trainer.save_model(output_dir = "./model/mygpt2")

